# Basel Precipitaion Forecast
In this project we train a prediction model for precipitation by using some data about weather.

## Abstract
- Suppose we have some data about a day and want to predict whether it will rain or not.
- First find patterns for T, H, W, P, C and also for Precipitation.
- Then train a model with different techniques to predict rainfall.
- Loading the essential libraries and then loading and reading the dataset.
- Visualizing the dataset using charts, graphs, etc., to understand the data better.
- Pre-processing the data using various data cleaning and manipulation techniques.
- Applying multiple machine learning algorithms to train the models and evaluate their performance using different metrics.

## Introduction
- Tell something about Basel city and importance of precipitation forecasting.
- Tell something about importance of up-to-date data of climate and new patterns.
- Tell something about advantages of forecasting and why it is important today.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn

## Dataset
##### [This is the link](https://www.meteoblue.com/en/weather/archive/export)
##### Talk about dataset
- how many rows and columns it consists of
- total number of records
- relation between the rows and columns of data
- various categories of data
##### Mention the attributes
- Temperature
- Pressure
- Cloud coverage
- Wind speed
- Humiidity
- Date
- Percipition

## Pre-Processing
What things should I do for cleaning data and make it ready for applying algorithms?
- Removing unnecessary features
- Normalizing

## Models
Let's have a look at each one in deep

### Linear Model
This is like ...
If we apply it, results:
###### SOME FIGUERS AND PLOTS TO SHOW THE RESULTS
Calculate performance, error, advantages and disadvantages

### Other Models
This is like ...
If we apply it, results:
###### SOME FIGUERS AND PLOTS TO SHOW THE RESULTS
Calculate performance, error, advantages and disadvantages

## Conclusion
- Forecasting is good.
- Undestand that these features have a stronger affect in result.
- This Model is better in performance.
- Add references if necessary.